In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import datetime
import scipy.stats as stats
import seaborn as sns
# import tsfel
from tsfresh.feature_extraction import extract_features, MinimalFCParameters, EfficientFCParameters
# import featuretools as ft

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [ ]:
# Sensor data
# Identify intervals with missing data in the raw signals
# Resmaple the time series with a smapling rate of 50 Hz
# Temoprally align the signals collected using different devices 


In [2]:
direc = "E:\\WS4PD_data"
os.chdir(direc)
# load data
demogra_data = pd.read_csv("Demographics_data.csv")
task_score = pd.read_csv("Task_scores_part_I.csv")
save_path = os.path.join(direc, "Figures", "Sensor_data","GENEActiv")

In [3]:
task_score['task_id'].unique()

array([  1,   2,   3,   4,   5,   6,  16,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [4]:
sum(task_score['task_id']==1)

252

In [ ]:
demogra_data.head()

In [ ]:
task_score.head()

In [ ]:
# checking the shortest duration
min(data.timestamp_end - data.timestamp_start)

In [ ]:
data_sensor

In [ ]:
# construct data for tsfresh
# testing GENEActiv, tremor, stndg task
device = 'GENEActiv'
visit= 1
task_code = 'stndg'
phenotype = 'tremor'
subject_ids = task_score.subject_id.unique()

data = task_score.loc[(task_score.visit == visit) &\
(task_score.task_code == task_code) &\
(task_score.phenotype == phenotype)]

sample_rate = 50
duration = 30 # select the first 30s 
dt = 1/sample_rate
t = np.round_([item * dt for item in range(duration*sample_rate)],2)

df =pd.DataFrame()

sample_id = 0
sample_score = []

not_number = pd.DataFrame()
# find sensor data
for sb in subject_ids:
    geneactive_hand = demogra_data.geneactive_hand.loc[demogra_data.subject_id == sb].values[0]
    body_segment = geneactive_hand+'UpperLimb'
    sensor_path = os.path.join(direc, device, sb, 'rawdata_day'+str(visit)+'.txt')
    data_sensor = pd.read_table(sensor_path)
    is_trialtype = (data.subject_id == sb) & (data.body_segment == body_segment)
    trial_idx = [i for i, val in enumerate(is_trialtype) if val]                   
    for trial in trial_idx:
        score = int(data['score'].values[trial])
        session = data['session'].values[trial]
        ts_start = data['timestamp_start'].values[trial]
        ts_end = data['timestamp_end'].values[trial]
        is_ts = (data_sensor.timestamp.values >= ts_start) & (data_sensor.timestamp.values <= ts_end)
        t0_idx = [i for i, val in enumerate(is_ts) if val][0]
        tw = sample_rate*duration
#         if sum(is_ts) > tw: # every trial is longer than trial duration (no missing time stamp)
        x = data_sensor['GENEActiv_X'].loc[t0_idx:t0_idx+tw-1].values
        y = data_sensor['GENEActiv_Y'].loc[t0_idx:t0_idx+tw-1].values
        z = data_sensor['GENEActiv_Z'].loc[t0_idx:t0_idx+tw-1].values
        is_number = [isinstance(xx, (int, float)) for xx in x]

        if sum(is_number) == sample_rate*duration: # all variables in x is a number
            id_xtime = np.array([sample_id]*len(t))
            d = {'id': id_xtime, 'time': t, 'x': x, 'y': y, 'z': z}
            df_trial = pd.DataFrame(d)
            df = pd.concat([df,df_trial])
            sample_score.append(score) 
            sample_id = sample_id + 1
        else:
            not_number_d = {'subject_id': sb, 'session': [session]}
            not_number_df = pd.DataFrame(not_number_d)
            not_number = pd.concat([not_number,not_number_df])

In [ ]:
not_number

In [ ]:
len(sample_score)

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
df2=df.astype(float)

In [ ]:
df2.dtypes

In [ ]:
# extract comprehensive features (default)
# extracted_features = extract_features(df2, column_id="id", column_sort="time")


In [ ]:
efficient_features = extract_features(df2, column_id="id", column_sort="time", default_fc_parameters=EfficientFCParameters())

In [ ]:
efficient_features

In [ ]:
sample_score = np.array(sample_score)

In [ ]:
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute

impute(efficient_features)
features_filtered = select_features(efficient_features, sample_score)

In [ ]:
features_filtered

In [ ]:
features_filtered.columns

In [ ]:
# Example session
device = 'GENEActiv'
subject_id = '6_BOS'
visit= 1
task_code = 'stndg'
session = 3
body_segment = 'RightUpperLimb'
phenotype = 'tremor'
most_affected_side = demogra_data['pd_most_affected_side'].loc[demogra_data.subject_id == subject_id]
is_trial = (task_score.subject_id == subject_id) & (task_score.visit == visit) &\
(task_score.task_code == task_code) & (task_score.session == session) &\
(task_score.phenotype == phenotype) & (task_score.body_segment == body_segment)
trial_idx = [i for i, val in enumerate(is_trial) if val][0]                    
score = task_score['score'].values[trial_idx]
ts_start = task_score['timestamp_start'].values[trial_idx]
ts_end = task_score['timestamp_end'].values[trial_idx]

# find sensor data
sensor_path = os.path.join(direc, device, subject_id, 'rawdata_day'+str(visit)+'.txt')
df = pd.read_table(sensor_path)


In [ ]:
df.head()

In [ ]:
is_ts = (df.timestamp.values >= ts_start) & (df.timestamp.values <= ts_end)
x = df['GENEActiv_X'].loc[is_ts].values
y = df['GENEActiv_Y'].loc[is_ts].values
z = df['GENEActiv_Z'].loc[is_ts].values
dt = round(df.timestamp[1] - df.timestamp[0],2)
t= range(len(x))*dt

In [ ]:
fig = plt.figure(figsize=(10,5))
plt.plot(t,x,'-b',t,y,'-g',t,z,'-r')
# # save figure
# plt.savefig(os.path.join(save_path,"6_BOS_RUL"))

In [ ]:
# construct data for GENEActiv, tremor, stndg task
device = 'GENEActiv'
visit= 1
task_code = 'stndg'
phenotype = 'tremor'
subject_ids = task_score.subject_id.unique()
data = task_score.loc[(task_score.visit == visit) &\
(task_score.task_code == task_code) &\
(task_score.phenotype == phenotype)]
sample_rate = 50
GA_X = ['nan']*len(data)
GA_Y = ['nan']*len(data)
GA_Z = ['nan']*len(data)

# GA_XYZ = [['nan']*3]*len(data)
# find sensor data
for sb in subject_ids:
    geneactive_hand = demogra_data.geneactive_hand.loc[demogra_data.subject_id == sb].values[0]
    body_segment = geneactive_hand+'UpperLimb'
    sensor_path = os.path.join(direc, device, sb, 'rawdata_day'+str(visit)+'.txt')
    data_sensor = pd.read_table(sensor_path)
    is_trialtype = (data.subject_id == sb) & (data.body_segment == body_segment)
    trial_idx = [i for i, val in enumerate(is_trialtype) if val]                   
    for trial in trial_idx:
        ts_start = data['timestamp_start'].values[trial]
        ts_end = data['timestamp_end'].values[trial]
        is_ts = (data_sensor.timestamp.values >= ts_start) & (data_sensor.timestamp.values <= ts_end)
        GA_X[trial] = data_sensor['GENEActiv_X'].loc[is_ts].values
        GA_Y[trial] = data_sensor['GENEActiv_Y'].loc[is_ts].values
        GA_Z[trial] = data_sensor['GENEActiv_Z'].loc[is_ts].values 
data['GENEActiv_X'] = GA_X
data['GENEActiv_Y'] = GA_Y
data['GENEActiv_Z'] = GA_Z
#         GA_XYZ[trial][0] = data_sensor['GENEActiv_X'].loc[is_ts].values
#         GA_XYZ[trial][1] = data_sensor['GENEActiv_Y'].loc[is_ts].values
#         GA_XYZ[trial][2] = data_sensor['GENEActiv_Z'].loc[is_ts].values    
# df_add = pd.DataFrame(data=GA_XYZ,columns=['GENEActiv_X', 'GENEActiv_Y', 'GENEActiv_Z'])
# data_completed = pd.concat([data,df_add], axis=1)

In [ ]:
data

In [ ]:
# check missing time stamp
interval = np.round_(np.diff(df.timestamp),2)
is_dt = interval == (1/sample_rate)
missing_ts = [i for i, val in enumerate(~is_dt) if val]                   
missing_ts

In [ ]:
device = 'GENEActiv'
visit= 1
task_code = 'stndg'
phenotype = 'tremor'
subject_ids = task_score.subject_id.unique()
data = task_score.loc[(task_score.visit == visit) &\
(task_score.task_code == task_code) &\
(task_score.phenotype == phenotype)]
sample_rate = 50

In [ ]:
# one subject
# find sensor data
subject_id = '6_BOS'
geneactive_hand = demogra_data.geneactive_hand.loc[demogra_data.subject_id == subject_id].values[0]
body_segment = geneactive_hand+'UpperLimb'
sensor_path = os.path.join(direc, device, subject_id, 'rawdata_day'+str(visit)+'.txt')
df = pd.read_table(sensor_path)

# find 
is_trialtype = (data.subject_id == subject_id) & (data.body_segment == body_segment)
trial_idx = [i for i, val in enumerate(is_trialtype) if val]                   


In [ ]:
ts_start = data['timestamp_start'].values[trial]
is_start = df.timestamp.values >= ts_start
t0_idx = [i for i, val in enumerate(is_start) if val][0]
t0_idx

In [ ]:
score = data['score'].values[trial]
ts_start = data['timestamp_start'].values[trial]
ts_end = data['timestamp_end'].values[trial]
is_ts = (df.timestamp.values >= ts_start) & (df.timestamp.values <= ts_end)
t0_idx = [i for i, val in enumerate(is_ts) if val][0]
tw = sample_rate*duration
if sum(is_ts) > tw:
    x = df['GENEActiv_X'].loc[t0_idx:t0_idx+tw-1].values
    y = df['GENEActiv_Y'].loc[t0_idx:t0_idx+tw-1].values
    z = df['GENEActiv_Z'].loc[t0_idx:t0_idx+tw-1].values
    sc = np.array([score]*len(t))
    d = {'id': sc, 'time': t, 'x': x, 'y': y, 'z': z}
    df = pd.DataFrame(d)
    df2 = pd.concat([df,df])

In [ ]:
dt = round(df.timestamp[1] - df.timestamp[0],2)
t= range(len(x))*dt

In [ ]:
fig = plt.figure(figsize=(10,5))
plt.plot(t,x,'-b',t,y,'-g',t,z,'-r')

In [ ]:
data.head()

In [ ]:
for trial in trial_idx:
    ts_start = data['timestamp_start'].values[trial]
    ts_end = data['timestamp_end'].values[trial]
    is_ts = (df.timestamp.values >= ts_start) & (df.timestamp.values <= ts_end)
    GA_XYZ[trial][0] = df['GENEActiv_X'].loc[is_ts].values
    GA_XYZ[trial][1] = df['GENEActiv_Y'].loc[is_ts].values
    GA_XYZ[trial][2] = df['GENEActiv_Z'].loc[is_ts].values
    

In [ ]:
# Clinical data
# 